In [1]:
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.chains import LLMChain, ConversationChain
from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI
import traceback
import random
import json
import pandas as pd
from dotenv import load_dotenv
import os
import torch


In [ ]:
# # Set a fixed seed for reproducibility
# random.seed(42)
# torch.manual_seed(42)

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-3.5-turbo", temperature=0.5)

Create prompt template to summarize the requirements 

In [ ]:
summary_json_format = """
{
    "summarized_requirements": [
        {
            "id": "Serial Number",
            "Requirement Name": "Give a name to the requirement in the format High_Level_Functionality_Low_Level_Functionality",
            "Description": "Provide a description of the requirement",
            "Priority": "Assign a priority to the requirement based on the criticality"
        },
        {
            "id": "Serial Number",
            "Requirement Name": "Give a name to the requirement in the format High_Level_Functionality_Low_Level_Functionality",
            "Description": "Provide a description of the requirement",
            "Priority": "Assign a priority to the requirement based on the criticality"
        },
    ]
}
"""

In [ ]:
summarize_template = """
As a product owner, your responsibility is to distill the provided requirements into a concise and comprehensive summary. \
Your task is to generate appropriate requirements based on the {requirements}, ensuring that all aspects mentioned in the \
requirements are adequately covered.

Generate as many requirements as necessary to encapsulate the specified functionality, ensuring that each aspect is \
addressed without duplication. Avoid adding any additional information beyond what is provided in the original requirements.

Finally, format your response according to the provided summary_json_format given below and use it as a guide.
{summary_json_format}
"""

In [ ]:
summarize_prompt = PromptTemplate(
    input_variables=["requirements","summary_response_json"],
    template=summarize_template
)

In [ ]:
summary_chain = LLMChain(
    llm = llm,
    prompt = summarize_prompt,
    output_key = "summarized_requirements",
    verbose = True
)

In [ ]:
tc_json_format = """
{
    "test_cases": [
        {
            "test_case_name": "TC001_Functionality_LowLevelFunctionality_TestCaseType_Positive",
            "pre_condition": "Condition to start the execution of the test case",
            "steps_to_execute": "Steps to execute the test case with each step numbered",
            "expected_result": "Desired result expected when the test case is executed"
        },
        {
            "test_case_name": "TC002_High_Level_Functionality_LowLevelFunctionality_TestCaseType_Negative",
            "pre_condition": "Condition to start the execution of the test case",
            "steps_to_execute": "Steps to execute the test case with each step numbered",
            "expected_result": "Desired result expected when the test case is executed"
        },
    ]
}

"""

In [ ]:
tc_generation_template = """
As an experienced software tester, your task is to create comprehensive test cases based on the provided \
user requirements outlined below. Utilize industry-standard testing techniques such as Boundary Value Analysis, \
Equivalence Partitioning, Decision Table Testing, and Use Case Testing to ensure thorough coverage of both positive \
and negative scenarios.

For each user requirement listed in {summarized_requirements}, systematically generate multiple test cases that validate the \
functionality under various conditions. Ensure to cover edge cases, error handling, and boundary scenarios to maximize \
test coverage.

Finally, format your response according to the tc_json_format template provided below and use it as a guide. Do not add \
any additional character on your own.
{tc_json_format}

Please proceed with your test case generation, keeping the above instructions in mind.

"""

In [ ]:
tc_generation_prompt = PromptTemplate(
    input_variables=["tc_json_format", "summarized_requirements"],
    template=tc_generation_template,
)

In [ ]:
tc_chain = LLMChain(
    llm = llm,
    prompt = tc_generation_prompt,
    output_key = "test_cases",
    verbose = True
)

In [ ]:
### Traceability Matrix

In [ ]:
tm_json_format = """
{
    "traceability_matrix": [
        {
            "User Requirement Id": "Requirement id from summarized_requirements e.g. 01",
            "User Requirement Name": "Name of the requirement from summarized_requirements",
            "Test Case Name": "Test case name from test_cases"
        },
        {
            "User Requirement Id": "Requirement id from summarized_requirements e.g. 02",
            "User Requirement Name": "Name of the requirement from summarized_requirements",
            "Test Case Name": "Test case name from test_cases"
        },
    ]
}

"""

In [ ]:
tm_generation_template = """"
Your role as a software tester entails the creation of a traceability matrix to establish clear links between user \
requirements and the corresponding test cases. This matrix serves as a crucial document for ensuring comprehensive \
test coverage and validating that all requirements are adequately addressed.

To construct the traceability matrix, meticulously match each user requirement listed in {summarized_requirements} \
to the relevant test case names listed in {test_cases}. If a single requirement is covered by multiple test cases, \
generate a distinct entry for each test case to maintain granularity.

Lastly, organize your response according to the specified tm_json_format template provided below. Do not add any additional character on your own.
Ensure that the resulting matrix accurately reflects the relationships between requirements and test cases. 

{tm_json_format}
"""

In [ ]:
tm_generation_prompt = PromptTemplate(
    input_variables=["summarized_requirements", "test_cases"],
    template=tm_generation_template,
)

In [ ]:
tm_chain = LLMChain(
    llm = llm,
    prompt = tm_generation_prompt,
    output_key = "traceability_matrix",
    verbose = True
)

In [ ]:
output_chain = SequentialChain(
    chains= [summary_chain, tc_chain, tm_chain],
    input_variables= ["requirements","summary_json_format", "tc_json_format", "tm_json_format"],
    output_variables= ["summarized_requirements", "test_cases", "traceability_matrix"],
    verbose = True
)

In [ ]:
input_file_path = "C:\Documents\AI\GenAI\E2E Test Case generator\input.txt"

In [ ]:
with open(input_file_path, 'r') as file:
    requirements = file.read()

In [ ]:
with get_openai_callback() as cb:
    response = output_chain(
        {
            "requirements": requirements,
            "summary_json_format": json.dumps(summary_json_format),
            "tc_json_format": json.dumps(tc_json_format),
            "tm_json_format": json.dumps(tm_json_format),
        }
    )

In [ ]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Propt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

In [ ]:
finalized_requirements = response.get("summarized_requirements")

In [ ]:
print(finalized_requirements)

In [ ]:
req_json_data=json.loads(finalized_requirements)

In [ ]:
req = req_json_data['summarized_requirements']

In [ ]:
req_df = pd.DataFrame(req)

In [ ]:
req_df.to_csv("requirements.csv", index=False)

In [ ]:
finalized_test_cases = response.get("test_cases")

In [ ]:
finalized_test_cases=finalized_test_cases.strip()
finalized_test_cases=finalized_test_cases.strip()

In [ ]:
print(finalized_test_cases)

In [ ]:
tc_json_data=json.loads(finalized_test_cases)

In [ ]:
tc = tc_json_data['test_cases']

In [ ]:
tc_df = pd.DataFrame(tc)

In [ ]:
tc_df.to_csv("test_cases.csv", index=False)

In [ ]:
finalized_traceability_matrix = response.get("traceability_matrix")

In [ ]:
tm_json_data=json.loads(finalized_traceability_matrix)

In [ ]:
tm = tm_json_data['traceability_matrix']

In [ ]:
tm_df = pd.DataFrame(tm)

In [ ]:
tm_df.to_csv("traceability_matrix.csv", index=False)